In [7]:
pip install pycaret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 483.9/483.9 kB 7.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 1.6 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.8/81.8 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 282.6/282.6 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 4.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 630.4/630.4 kB 8.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.8/85.8 MB 3.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.6/235.6 kB 4.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.4/15

In [1]:
import pandas as pd

# 데이터 불러오기
match_data = pd.read_csv('/Users/lhe339/Documents/GitHub/nextorial/data/데이터 분석가_과제_Type A/match_data.csv')

In [2]:
def calculate_team_features(data):
    return data.groupby(['matchid', 'teamid']).agg({
        'mmr': 'mean',
        'winstreak': 'mean',
        'losestreak': 'mean',
        'recentwinprob': 'mean'
    }).reset_index()

# match_data에 대한 전처리
match_team_features = calculate_team_features(match_data)

# 원래 match_data와 merge하여 각 유저 옆에 팀의 평균 특성값 추가하기
match_data = match_data.merge(match_team_features, on=['matchid', 'teamid'], suffixes=('', '_team_avg'))


In [4]:
def guild_avg(df):
    # 'guildid'에서 NaN 값을 처리합니다. 이 예에서는 'NoGuild'로 대체합니다.
    df['guildid'].fillna('NoGuild', inplace=True)

    # 티어를 숫자로 변환합니다.
    tier_mapping = {'unranked': 0, 'bronze': 1, 'silver': 2, 'gold': 3, 'platinum': 4, 'diamond': 5, 'master': 6}
    df['tier_numeric'] = df['tier'].map(tier_mapping)

    # 길드별로 데이터를 그룹화하고 그룹별로 평균을 계산합니다.
    guild_avg = df.groupby('guildid')[['mmr', 'winstreak', 'recentwinprob', 'accumatches', 'tier_numeric']].mean()

    # 새로운 컬럼 이름을 생성합니다.
    new_columns = {col: f'{col}_guild_avg' for col in guild_avg.columns}
    guild_avg.rename(columns=new_columns, inplace=True)

    # 원본 데이터셋에 계산된 길드 성능 지표를 병합합니다.
    df = pd.merge(df, guild_avg, on='guildid', how='left')
    
    return df


match_data = guild_avg(match_data)